In [6]:
import numpy as np
import random

Some Helper functions for later

In [4]:
def generate_ground_plane(width: int, length: int) -> str:
    """Generates the ground plane in URDF format based on width and length."""
    height = 0.1  # Thin height to represent the ground plane
    
    ground_plane_str = f"""
        <link name="ground_plane">
            <inertial>
                <origin xyz="0 0 {-height/2}" rpy="0 0 0"/>
                <mass value="0"/>
                <inertia ixx="1" ixy="0" ixz="0" iyy="1" iyz="0" izz="1"/>
            </inertial>
            <collision>
                <geometry>
                    <box size="{width} {length} {height}"/>
                </geometry>
            </collision>
            <visual>
                <geometry>
                    <box size="{width} {length} {height}"/>
                </geometry>
                <material name="ground_color">
                    <color rgba="0.3 0.7 0.3 1.0"/> <!-- Greenish color -->
                </material>
            </visual>
        </link>
    """
    return ground_plane_str

def generate_terrain(width: int, length: int, amplitude: float) -> str:
    """Generates the terrain in URDF format based on width, length, and amplitude."""
    x = np.linspace(-width/2, width/2, width)
    y = np.linspace(-length/2, length/2, length)
    X, Y = np.meshgrid(x, y)
    Z = amplitude * np.sin(np.sqrt(X**2 + Y**2))

    # Convert the grid to a list of vertices and triangles
    vertices = list(zip(X.flatten(), Y.flatten(), Z.flatten()))
    triangles = []
    for i in range(width-1):
        for j in range(length-1):
            triangles.append((i*length + j, (i+1)*length + j, (i+1)*length + j+1))
            triangles.append((i*length + j, (i+1)*length + j+1, i*length + j+1))

    # Save as an obj file
    obj_filename = "terrain.obj"
    with open(obj_filename, 'w') as file:
        for v in vertices:
            file.write(f"v {v[0]} {v[1]} {v[2]}\n")
        for t in triangles:
            file.write(f"f {t[0]+1} {t[1]+1} {t[2]+1}\n")

    terrain_str = f"""
        <link name="terrain">
            <inertial>
                <origin xyz="0 0 0" rpy="0 0 0"/>
                <mass value="0"/>
                <inertia ixx="1" ixy="0" ixz="0" iyy="1" iyz="0" izz="1"/>
            </inertial>
            <collision>
                <geometry>
                    <mesh filename="{obj_filename}"/>
                </geometry>
            </collision>
            <visual>
                <geometry>
                    <mesh filename="{obj_filename}"/>
                </geometry>
            </visual>
        </link>
    """
    return terrain_str

def generate_tree(id: int) -> str:
    """Generates the tree in URDF format."""
    trunk_height = random.uniform(1.0, 3.0)  # Random height for the trunk
    trunk_radius = random.uniform(0.2, 0.5)  # Random radius for the trunk
    foliage_radius = trunk_radius * 2  # The radius of the foliage sphere
    total_height = trunk_height + foliage_radius * 2

    # Approximate inertia for a cylinder (trunk)
    trunk_mass = 10  # Arbitrary mass for the trunk
    inertia_ixx = (trunk_mass * trunk_radius**2) / 2
    inertia_iyy = inertia_ixx
    inertia_izz = trunk_mass * trunk_radius**2

    tree_str = f"""
        <link name="tree_{id}">
            <!-- Trunk (cylinder) -->
            <collision name="trunk_collision">
                <origin xyz="0 0 {trunk_height/2}" rpy="0 0 0"/>
                <geometry>
                    <cylinder length="{trunk_height}" radius="{trunk_radius}"/>
                </geometry>
            </collision>
            <visual name="trunk_visual">
                <origin xyz="0 0 {trunk_height/2}" rpy="0 0 0"/>
                <geometry>
                    <cylinder length="{trunk_height}" radius="{trunk_radius}"/>
                </geometry>
                <material name="brown">
                    <color rgba="0.545 0.271 0.075 1.0"/> <!-- Brown color -->
                </material>
            </visual>
            
            <!-- Foliage (sphere) -->
            <collision name="foliage_collision">
                <origin xyz="0 0 {trunk_height + foliage_radius}" rpy="0 0 0"/>
                <geometry>
                    <sphere radius="{foliage_radius}"/>
                </geometry>
            </collision>
            <visual name="foliage_visual">
                <origin xyz="0 0 {trunk_height + foliage_radius}" rpy="0 0 0"/>
                <geometry>
                    <sphere radius="{foliage_radius}"/>
                </geometry>
                <material name="dark_green">
                    <color rgba="0 0.5 0 1.0"/> <!-- Dark green color -->
                </material>
            </visual>

            <inertial>
                <origin xyz="0 0 {total_height/2}" rpy="0 0 0"/>
                <mass value="{trunk_mass}"/> <!-- Using the trunk mass for the whole tree, but this can be refined further -->
                <inertia ixx="{inertia_ixx}" ixy="0" ixz="0" iyy="{inertia_iyy}" iyz="0" izz="{inertia_izz}"/>
            </inertial>
        </link>
    """
    return tree_str


def generate_fixed_joint(child_name: str, x: float, y: float) -> str:
    """Generates a fixed joint that connects the specified child link to the root link."""
    joint_str = f"""
        <joint name="joint_{child_name}" type="fixed">
            <parent link="root"/>
            <child link="{child_name}"/>
            <origin xyz="{x} {y} 0" rpy="0 0 0"/>
        </joint>
    """
    return joint_str

def generate_root() -> str:
    """Generates the root link in URDF format."""
    root_str = """
        <link name="root">
            <inertial>
                <origin xyz="0 0 0" rpy="0 0 0"/>
                <mass value="0"/>
                <inertia ixx="1" ixy="0" ixz="0" iyy="1" iyz="0" izz="1"/>
            </inertial>
        </link>
    """
    return root_str

## Parameters

You can adjust these parameters to define how the world is generated

In [8]:
OUTPUT_FILE="environment.urdf"  # Copy this to assets when to your liking
WIDTH=50
LENGTH=50
AMPLITUDE=5
TREES=25

## Build

Build the URDF File

In [9]:
file = open(OUTPUT_FILE, 'w')    
file.write('<?xml version="1.0" ?>\n')
file.write('<robot name="environment">\n')

# Generate root link
file.write(generate_root())

# Generate ground plane
file.write(generate_ground_plane(WIDTH, LENGTH))
file.write(generate_fixed_joint("ground_plane", 0, 0))

# Generate terrain
terrain_str = generate_terrain(WIDTH, LENGTH, AMPLITUDE)
file.write(terrain_str)
file.write(generate_fixed_joint("terrain", 0, 0))

# Generate trees
for i in range(TREES):
    x = random.uniform(-WIDTH/2, WIDTH/2)
    y = random.uniform(-LENGTH/2, LENGTH/2)
    tree_str = generate_tree(i)
    file.write(tree_str)
    file.write(generate_fixed_joint(f"tree_{i}", x, y))


file.write('</robot>')
file.close()

In [10]:
!cp -v environment.urdf ../gym_pybullet_drones/assets/

environment.urdf -> ../gym_pybullet_drones/assets/environment.urdf
